In [ ]:
!wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip && unzip v2_Annotations_Train_mscoco && \
wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip && unzip v2_Questions_Train_mscoco

--2020-11-07 10:50:42--  https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Train_mscoco.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.106.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.106.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21708861 (21M) [application/zip]
Saving to: ‘v2_Annotations_Train_mscoco.zip’

v2_Annotations_Trai 100%[===================>]  20.70M  44.4MB/s    in 0.5s    

2020-11-07 10:50:43 (44.4 MB/s) - ‘v2_Annotations_Train_mscoco.zip’ saved [21708861/21708861]

Archive:  v2_Annotations_Train_mscoco.zip
  inflating: v2_mscoco_train2014_annotations.json  
--2020-11-07 10:50:45--  https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Train_mscoco.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.106.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.106.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7239401 (6.9M) [application/zip]
Saving to: ‘

In [ ]:
import json
annotations = json.load(open("v2_mscoco_train2014_annotations.json"))
questions = json.load(open("v2_OpenEnded_mscoco_train2014_questions.json"))

questions_id = {q['question_id']: q['question'] for q in questions['questions']}

In [ ]:
from collections import Counter, defaultdict

top_answers = Counter()
answers_questions = defaultdict(list)

for annotation in (annotations['annotations']):
  question_id =  annotation['question_id']
  image_id =  annotation['image_id']
  # example Counter({'down': 7, 'at table': 1, 'skateboard': 1, 'table': 1})
  answers = Counter([answer['answer'] for answer in annotation['answers']])
  # We take the most chosen answer for the question, and update the top_answers counter
  top_answer, _ = answers.most_common()[0]
  top_answers.update([top_answer])
  # fetch question
  question = questions_id[question_id]
  # recording question and image id for this answer
  answers_questions[top_answer].append([question, int(image_id), int(question_id)])

In [ ]:
import urllib.request
import os
from collections import defaultdict
from tqdm import tqdm

top100_answers = top_answers.most_common(100)
print(top100_answers)
answer_list = [ans for ans,_ in top100_answers]

image_question = defaultdict(list)
label_question = defaultdict(list)
question_label = {}

os.makedirs("data", exist_ok=True)
# From top 100 answers, we populate our dataset
for ta in tqdm(top100_answers):
  answer, _ = ta
  questions = answers_questions[answer]
  # if number of questions is over 1000, we take 1000, otherwise min(500, len(questions)
  num_questions = 1000 if len(questions) > 1000 else min(500, len(questions))

  for qa in questions[:num_questions]:
    question, image_id, question_id = qa
    # Downloading questions
    # filename = 'COCO_train2014_'+ str(image_id).zfill(12) + '.jpg'
    # url = os.path.join('https://vqa_mscoco_images.s3.amazonaws.com/train2014/', filename)
    # if os.path.exists(os.path.join('data', filename)):
    #   urllib.request.urlretrieve(url, os.path.join('data', filename))
    # Filling dict
    image_question[image_id].append((question_id, question))
    label_question[answer].append(question_id)
    question_label[question_id] = answer_list.index(answer)
    
print(len(image_question), 'image_question', list(image_question.items())[:5])
print(len(question_label), 'question_label', list(question_label.items())[:5])


 17%|█▋        | 17/100 [00:00<00:00, 169.66it/s]

[('no', 84572), ('yes', 82981), ('2', 12514), ('1', 12257), ('white', 9018), ('3', 6269), ('red', 5352), ('blue', 5200), ('black', 5176), ('0', 5001), ('4', 4370), ('green', 3774), ('brown', 3595), ('yellow', 2834), ('5', 2341), ('gray', 1990), ('nothing', 1860), ('left', 1684), ('right', 1668), ('frisbee', 1642), ('6', 1623), ('baseball', 1607), ('tennis', 1496), ('wood', 1470), ('orange', 1408), ('none', 1316), ('bathroom', 1219), ('pizza', 1218), ('pink', 1128), ('kitchen', 1113), ('7', 990), ('8', 966), ('man', 951), ('cat', 937), ('grass', 904), ('skateboarding', 897), ('dog', 895), ('water', 884), ('skiing', 882), ('silver', 821), ('black and white', 817), ('10', 801), ('kite', 790), ('surfing', 747), ('horse', 704), ('skateboard', 696), ('living room', 689), ('phone', 656), ('snow', 643), ('wii', 642), ('giraffe', 638), ('surfboard', 624), ('woman', 607), ('cake', 605), ('sunny', 594), ('broccoli', 590), ('eating', 587), ('table', 584), ('hat', 576), ('apple', 576), ('stop', 562

100%|██████████| 100/100 [00:00<00:00, 436.34it/s]

36042 image_question [(393224, [(393224000, 'Is the man smiling?'), (393224002, 'Does his tie pair well with his suit?'), (393224003, 'Does the man look happy?'), (393224004, 'Is this photo in color?'), (393224005, 'Is the man wearing a plain tie?')]), (524297, [(524297002, 'Are the men on the sidewalk?'), (524297003, 'Was this photo taken recently?'), (524297000, 'Judging from the dress, was this taken in a Latin American country?'), (524297001, 'What colors are shown in this picture?')]), (393227, [(393227004, "Is the man's visor providing his face enough protection?"), (393227000, 'Does the guy have a tattoo?'), (393227002, "How many tattoos can be seen on this man's body?"), (393227003, 'What color is his hat?'), (393227001, 'What is this man riding on?')]), (131074, [(131074003, 'Is the curtain patterned?'), (131074002, 'Are the walls done in a summery color?'), (131074005, 'Are the windows the same size?'), (131074001, 'How many pieces of furniture which are used for sleeping are

In [ ]:
# Resizing
import os
from PIL import Image
import sys
from tqdm import tqdm

outdir= "image124"
os.makedirs(outdir, exist_ok=True)
for k in tqdm(image_question.keys()):
  filename = "COCO_train2014_" + str(k).zfill(12) + ".jpg"
  image_path = "data/train2014/"+filename
  if not os.path.exists(image_path):
    print(image_path)
    sys.exit(1)
  
  outfile = os.path.join(outdir,k+'.jpg')
  if not os.path.exists(outfile):
    try:
      image = Image.open(image_path)
      image = image.resize((124,124), Image.ANTIALIAS)
      image.save(outfile, "JPEG")
    except OSError:
      print(outfile)

  0%|          | 0/36042 [00:00<?, ?it/s]

data/train2014/COCO_train2014_000000393224.jpg


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from google.colab import files

train = []
val = []
test = []

# generating balanced splits
# for each label, split question 80 10 10
import random
for k,v in  label_question.items():
  random.shuffle(v)
  l = len(v)
  train += (v[:int(0.8*l)])
  test +=  (v[int(0.8*l):int(0.9*l)])
  val += (v[int(0.9*l):])

random.shuffle(train)
random.shuffle(val)
random.shuffle(test)

for fn, ids in [["train.csv", train], ["val.csv", val], ["answer_key.csv", test], ["sample_submission.csv", test]]:
  lines = []
  for id in ids:
    # for sample_submission, random label
    if 'sample_submission' in fn:
      lines.append(str(id) +','+str(random.randint(0, len(label_question.keys())-1)))
    else:
      lines.append(str(id) +','+str(question_label[id]))
  
  open(fn, "w+").write('id,label\n'+'\n'.join(map(str, lines)))
  files.download(fn) 

# test file without label
open("test.csv", "w+").write('id,label\n'+'\n'.join(map(str, test)))
files.download('test.csv')
open("answer_list.txt", "w+").write('\n'.join(map(str, answer_list)))
files.download('answer_list.txt') 

json.dump(image_question, open('image_question.json', 'w'))
files.download('image_question.json') 


# question_label_test = {}
# for id in test:
#   v = question_label.pop(id, None)
#   assert v is not None
#   question_label_test[id] = v

# json.dump(question_label, open('question_label.json', 'w'))
# json.dump(question_label_test, open('question_label_test.json', 'w'))

# files.download('image_question.json') 
# files.download('question_label.json') 
# files.download('question_label_test.json') 
# files.download('train.txt') 
# files.download('val.txt') 
# files.download('test.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>